In [2]:

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from time import sleep
import datetime
import pandas as pd
from bs4 import BeautifulSoup
from splinter import Browser
from pymongo import MongoClient
import json

# edit these three variables
user = 'realDonaldTrump'
start = datetime.datetime(2017, 1, 11)  # year, month, day
end = datetime.datetime(2018, 12, 4)  # year, month, day

# only edit these if you're having problems

# don't mess with this stuff
days = (end - start).days + 1
user = user.lower()

driver = webdriver.Chrome("C:/Users/PhoenixJauregui/Documents/trump/trump/chromedriver2.exe")

executable_path = {'executable_path': 'C:/Users/PhoenixJauregui/Documents/trump/trump/chromedriver2.exe'}
browser = Browser('chrome', **executable_path, headless=False)

delay = 1

def format_day(date):
    day = '0' + str(date.day) if len(str(date.day)) == 1 else str(date.day)
    month = '0' + str(date.month) if len(str(date.month)) == 1 else str(date.month)
    year = str(date.year)
    return '-'.join([year, month, day])

def form_url(since, until):
    p1 = 'https://twitter.com/search?f=tweets&vertical=default&q=from%3A'
    p2 =  user + '%20since%3A' + since + '%20until%3A' + until + 'include%3Aretweets&src=typd'
    return p1 + p2

def increment_day(date, i):
    return date + datetime.timedelta(days=i)

def concat(words):
    concatenated = ""
    for i in range(len(words)):
        if concatenated == "":
            concatenated = words[i]
        else:    
            concatenated = concatenated +", " + words[i]
    return concatenated

dates = []
tweet_list = []   
trends = []
aux = []
increment = 0

for day in range(days):
    d1 = format_day(increment_day(start, 0))
    d2 = format_day(increment_day(start, 1))
    url = form_url(d1, d2)
    print(d1)
    driver.get(url)
    browser.visit(url)
    sleep(delay)
    
    # Retrieve all elements that contain tweet information
    html = browser.html
    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')    
    try:
        elements = driver.find_elements_by_css_selector('.ProfileTweet-actionCountForPresentation')
        for element in elements:
            if element == '':
                aux.append('0')
            else:
                aux.append(element.text)
        tweets = soup.find_all('p', class_= "TweetTextSize js-tweet-text tweet-text")
        for tweet in tweets:    
            tweet2 = str(tweet.find_all(text=True, recursive=False)).replace(',', '').replace('\'','').replace('â€','').replace(':','').replace('\\','').replace("[","").replace("]","").replace("â€œ","")
            tweet_list.append(tweet2)
            dates.append(d1)
            
            increment += 1
            print("processing tweet " + str(increment))
            
            try:
                links = tweet.find_all('a')
                links2 = concat([link.find("b").text for link in links])
                trends.append(links2)
            except:
                trends.append("")
    except NoSuchElementException:
        print('no tweets on this day')
        
    start = increment_day(start, 1)
    
retweets = aux[1::5]
comments = aux[0::5]
favs = aux[3::5]


2017-01-11
processing tweet 1
processing tweet 2
processing tweet 3
processing tweet 4
processing tweet 5
processing tweet 6
2017-01-12
processing tweet 7
processing tweet 8
processing tweet 9
processing tweet 10
processing tweet 11
processing tweet 12
2017-01-13
processing tweet 13
processing tweet 14
processing tweet 15
processing tweet 16
processing tweet 17
processing tweet 18
processing tweet 19
processing tweet 20
2017-01-14
processing tweet 21
processing tweet 22
processing tweet 23
2017-01-15
processing tweet 24
processing tweet 25
processing tweet 26
processing tweet 27
processing tweet 28
processing tweet 29
processing tweet 30
processing tweet 31
2017-01-16
processing tweet 32
processing tweet 33
processing tweet 34
2017-01-17
processing tweet 35
processing tweet 36
processing tweet 37
processing tweet 38
processing tweet 39
processing tweet 40
processing tweet 41
processing tweet 42
processing tweet 43
processing tweet 44
processing tweet 45
2017-01-18
processing tweet 46
p

In [9]:
'''favs=[float(i.replace('K', '')) * 1000 for i in favs]
retweets = [float(i.replace('K', '')) * 1000 for i in retweets]
comments = [float(i.replace('K', '')) * 1000 for i in comments]'''


twitter = pd.DataFrame({
'Date': dates,
'Tweets': tweet_list,
'Trending': trends,
'Retweets': retweets,
'Favs': favs,
'Comments': comments,
})

twitter["Date"] = pd.to_datetime(twitter["Date"].values, infer_datetime_format=True)

twitter = twitter[twitter['Favs'] > 100]
twitter = twitter.set_index("Date")
twitter_group = twitter.groupby(pd.TimeGrouper("3d"))["Favs"].mean()

twitter.to_csv("tweets3.csv")

polls = pd.read_csv("https://projects.fivethirtyeight.com/trump-approval-data/approval_polllist.csv")
polls["enddate"] = pd.to_datetime(polls["startdate"].values)
polls_filtered = polls.set_index("enddate")['2017-1-11':'2018-12-04']
polls_group = polls_filtered.groupby(pd.TimeGrouper("3d"))['approve'].mean()

complete = pd.DataFrame(twitter_group).join(polls_group,how="right")
complete_final = complete.reset_index()
complete_final = complete_final.dropna()

dat = complete_final.to_dict(orient = "records")
dat2 = twitter.to_dict(orient = "records")
dat3 = polls_filtered.to_dict(orient = "records")

mongo = MongoClient('ds017678.mlab.com', 17678)
db = mongo['heroku_sjm9g8xc']
db.authenticate('heroku_sjm9g8xc', '4mvq9ej8mp5hlvqdqbgc53d2bf')

scraped = db.compare
'''scraped2 = db.twitter
scraped3 = db.polls'''

for pushed in dat:
    scraped.insert(pushed)

'''for pushed in dat2:
    scraped2.insert(pushed)

for pushed in dat3:
    scraped3.insert(pushed)'''

print(favs)

C:\Users\PhoenixJauregui\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\ipykernel\__main__.py:19: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
C:\Users\PhoenixJauregui\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\ipykernel\__main__.py:26: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
C:\Users\PhoenixJauregui\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\ipykernel\__main__.py:45: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


[100000.0, 106000.0, 82000.0, 61000.0, 41000.0, 86000.0, 80000.0, 118000.0, 86000.0, 66000.0, 48000.0, 108000.0, 102000.0, 120000.0, 62000.0, 53000.0, 50000.0, 50000.0, 68000.0, 75000.0, 63000.0, 71000.0, 81000.0, 78000.0, 65000.0, 65000.0, 119000.0, 73000.0, 73000.0, 113000.0, 88000.0, 151000.0, 54000.0, 42000.0, 97000.0, 39000.0, 59000.0, 62000.0, 52000.0, 55000.0, 99000.0, 113000.0, 43000.0, 54000.0, 57000.0, 128000.0, 100000.0, 62000.0, 53000.0, 54000.0, 54000.0, 45000.0, 35000.0, 62000.0, 61000.0, 53000.0, 150000.0, 141000.0, 117000.0, 77000.0, 61000.0, 86000.0, 81000.0, 55000.0, 151000.0, 163000.0, 139000.0, 208000.0, 171000.0, 85000.0, 103000.0, 254000.0, 106000.0, 157000.0, 168000.0, 274000.0, 377000.0, 205000.0, 200000.0, 120000.0, 167000.0, 124000.0, 94000.0, 103000.0, 146000.0, 71000.0, 124000.0, 100000.0, 121000.0, 181000.0, 195000.0, 152000.0, 95000.0, 105000.0, 69000.0, 68000.0, 108000.0, 99000.0, 120000.0, 50000.0, 42000.0, 143000.0, 103000.0, 140000.0, 105000.0, 54000.0